### Bot 2:

A bot of your own design that uses the data from the detection square.

Split the ship into *k* by *k* tiles. Use Prim's algorithm to construct a minimum spanning tree connecting all of the tiles. First, assign the bot's initial location $v_s$ to the tile containing it $T_s$. Suppose you are exploring the neighbors of a tile $T_1$ (with vertex $v_1$) and come across a tile $T_2$. Then, find the vertex in $T_2$ that is closest to $v_1$, and assign it to $T_2$. Now, we have a minimum spanning tree that will guarantee we scan every vertex.

Construct a path using the vertices in the MST. Once you detect the leak, use BFS on the leaky squares to find the exact location of the leak.

In [ ]:
class Vertex():
    def __init__(self, tile_coord, cell_coord=None):
        self.tile_coord = tile_coord
        self.cell_coord = cell_coord
        self.has_assigned_coord = cell_coord is not None

    def __str__(self):
        return f"V({self.tile_coord})"

    def assign_coord(self, cell_coord):
        self.cell_coord = cell_coord
        self.has_assigned_coord = True

    def has_coord(self):
        return self.has_assigned_coord

In [ ]:
MST_DX = [0, 0, 1, 1, 1, -1, -1, -1]
MST_DY = [-1, 1, -1, 0, 1, -1, 0, 1]
class MST():
    def __init__(self, ship, k, init_pos):
        self.ship = ship
        self.k = k
        self.tile_D = k + 1
        #self.vertex_id = 0
        self.edges = dict()
        self.init_pos = init_pos

        self.D = math.ceil(ship.D /self.tile_D)
        self.vertices = [[None for _ in range(self.D)] for _ in range(self.D)]
        # self.ind_board = [[-1 for _ in range(self.D)] for _ in range(self.D)]
        
        self.num_tiles = 0
        self.create_MST()
    
    def valid_vertex(self, r, c):
        if r < 0 or c < 0:
            return False
        if r >= self.D or c >= self.D:
            return False
        return True

    def get_vertex(self, tile_coord):
        r, c = tile_coord
        if self.valid_vertex(r, c):
            return self.vertices[r][c]
        return None

    def create_MST(self):
        self.init_vertices()
        self.prim()

    def init_vertices(self):
        init_vertex = self.cell_to_tile(self.init_pos)
        self.start_vertex = None
        for tile_row in range(self.D): # tile rows
            for tile_col in range(self.D): # tile columns
                if not self.tile_visitable(tile_row, tile_col):
                    #print("not visitable", tile_row, tile_col)
                    continue

                vertex = Vertex(tile_coord=(tile_row, tile_col))
                if init_vertex == (tile_row, tile_col): # if 
                    vertex.assign_coord(self.init_pos)
                    self.start_vertex = vertex
                    
                self.vertices[tile_row][tile_col] = vertex
                self.num_tiles += 1
        if self.start_vertex == None:
            print(f"WARNING: no start vertex initialized {self.init_pos} {init_vertex}")

    def is_valid(self, r, c):
        """Determine if a row x col is within the bounds of the board"""
        return r >= 0 and c >= 0 and r < self.D and c < self.D

    def tile_visitable(self, tile_row, tile_col):
        """Check if given tile has at least one open cell"""
        start_row = tile_row * self.tile_D
        start_col = tile_col * self.tile_D
        # r, c = cell displacement within tile
        for dr in range(self.tile_D): 
            for dc in range(self.tile_D):
                r = start_row + dr
                c = start_col + dc
                if self.ship.is_valid(r, c):
                    if self.ship.board[r][c] == Cell.OPEN:
                        return True
        #print(f"Tile ({tile_row}, {tile_col}) is not visitable")
        return False

    def get_tile_neighbors(self, tile_coord):
        """Get all the valid neighbors of tile"""
        tile_row, tile_col = tile_coord
        
        neighbor_coords = []
        for i in range(4):
            nr = DY[i] + tile_row
            nc = DX[i] + tile_col
            if self.is_valid(nr, nc) and self.vertices[nr][nc] != None:
                neighbor_coords.append((nr, nc))

        return neighbor_coords

    def add_edge(self, A, B, path):
        if A not in self.edges:
            self.edges[A] = deque()
        if B not in self.edges:
            self.edges[B] = deque()

        reversed_path = copy.deepcopy(path)
        reversed_path.reverse()
        
        self.edges[A].append((B, path))
        self.edges[B].append((A, reversed_path))

    def get_edge(self, A, B):
        # print(A, B)
        for temp_B, path in self.edges[A]:
            if temp_B == B:
                return A, B, path
        return None, None, None

    def tile_manhattan(self, A, B):
        r_A, c_A = A
        r_B, c_B = B
        return abs(r_A - r_B) + abs(c_A - c_B)

    def prim(self):
        open = PriorityQueue()
        closed = set()
        # dist to parent, vertex, parent
        # open.put((0, self.start_vertex, None))
        curr = self.start_vertex
        start_tile = curr.tile_coord
        neighbors = self.get_tile_neighbors(curr.tile_coord)
        neighbor_path = self.tile_bfs(curr.cell_coord, neighbors)

        closed.add(start_tile)
        for n_coord in neighbor_path.keys():
            r, c = n_coord

            neighbor = self.vertices[r][c]
            path = neighbor_path[n_coord]
            len_path = len(path) - 1

            if not neighbor.has_coord():
                neighbor.assign_coord(path[-1])
            open.put((len_path, path, curr, neighbor))

            # self.add_edge(curr, neighbor, path)
            # print(len_path, n_coord, path)
        while open.qsize() > 0:
            # print("ASDF", self.start_vertex.cell_coord, self.start_vertex.has_assigned_coord)

            edge = open.get()
            len_path, path, curr, neighbor = edge

            if neighbor.tile_coord in closed:
                continue

            self.add_edge(curr.tile_coord, neighbor.tile_coord, path)
            closed.add(neighbor.tile_coord)
            # print("asdf", self.start_vertex.cell_coord, self.start_vertex.has_assigned_coord)

            curr = neighbor
            neighbors = self.get_tile_neighbors(curr.tile_coord)
            neighbor_path = self.tile_bfs(curr.cell_coord, neighbors)
            for n_coord in neighbor_path.keys():
                r, c = n_coord

                neighbor = self.vertices[r][c]
                path = neighbor_path[n_coord]
                len_path = len(path) - 1
                if not neighbor.has_coord():
                    neighbor.assign_coord(path[-1])
                open.put((len_path, path, curr, neighbor))

        if True:
            ct = 0
            for key in self.edges:
                ct += len(self.edges[key])
            # print(len(self.edges.keys()), ct)
        # TODO: finish method

    def cell_to_tile(self, cell_coord): 
        r, c = cell_coord
        return (math.floor(r / self.tile_D), math.floor(c / self.tile_D))
    
    # Input vertex ind; get the coordinates of assigned point
    def tile_to_cell(self, r, c):
        vertex = self.vertices[r][c]
        if vertex == None:
            return None
        return vertex.cell_coord
    
    def cell_in_tile(self, cell_coord, goal_tile):
        cell_tile = self.cell_to_tile(cell_coord)
        return cell_tile == goal_tile
    
    def cell_bfs(self, src, goal):
        """Shortest Path from SRC to GOAL -> Returns GOAL, PATH"""

        # Add support for multiple goal states
        if isinstance(goal, tuple):
            state = goal
            goal = set()
            goal.add(state)
        
        queue = deque()
        queue.append((src, deque()))
        vis = set()
        while queue: 
            curr, path = queue.popleft()
            new_path = path.copy()
            new_path.append(curr)
            vis.add(curr)

            if curr in goal:
                return (curr, new_path)
              
            r, c = curr
            for i in range(4):
                nr = DY[i] + r
                nc = DX[i] + c
                if self.ship.is_valid(nr, nc) and (nr, nc) not in vis and self.ship.board[nr][nc] != Cell.BLOCKED:
                    queue.append(((nr, nc), new_path))   

        return (None, deque())   

    def tile_bfs(self, src, goal):
        """Shortest Path from vertex A to tile B"""

        # Add support for multiple goal states
        if isinstance(goal, tuple):
            state = goal
            goal = set()
            goal.add(state)
        
        paths_to_neighbors = dict()
        
        queue = deque()
        queue.append((src, deque()))
        vis = set()
        while queue: 
            curr, path = queue.popleft()
            new_path = path.copy()
            new_path.append(curr)
            vis.add(curr)

            curr_tile = self.cell_to_tile(curr)
            r_tile, c_tile = curr_tile
            if curr_tile in goal:
                if not self.vertices[r_tile][c_tile].has_coord() or curr == self.vertices[r_tile][c_tile].cell_coord:
                    # print("asdf")

                    # print(src, self.vertices[r_tile][c_tile].cell_coord, path)
                    self.check_cell_traversal_order(new_path)
                    paths_to_neighbors[curr_tile] = new_path
                    goal.remove(curr_tile)
            # elif curr_tile != start_tile:
            #     continue
            if not goal:
                return paths_to_neighbors

            r, c = curr
            for i in range(4):
                nr = DY[i] + r
                nc = DX[i] + c
                if self.ship.is_valid(nr, nc) and (nr, nc) not in vis and self.ship.board[nr][nc] != Cell.BLOCKED:
                    queue.append(((nr, nc), new_path))   
        
        if False:
            if goal:
                for tile in goal:
                    paths_to_neighbors[tile] = None

        return paths_to_neighbors
    
    def check_cell_traversal_order(self, order):
        for i in range(len(order) - 1):
            a = order[i]
            b = order[i + 1]
            if manhattan(a, b) > 1:
                print(f"error in cell traversal {a} {b}")
                # TODO: finish
        return 

    def full_traversal_order(self, curr, num_nodes, visited, path):
        if len(visited) == num_nodes:
            ### print("done")
            return path
        visited.add(curr)
        
        #path = path + [curr]
        to_print = False
        printed = False
        for neighbor, path_to in self.edges[curr]:
            if neighbor not in visited:
                if to_print: print("->", curr)
                path.append(curr)
                printed = True
                path = self.full_traversal_order(neighbor, num_nodes, visited, path)
                #path += [curr]
            #print("ASDF", curr)
        if to_print:
            if not printed:
                print("->", curr)
            else: print("<-", curr)
        path.append(curr)
        return path
    
    def fast_traversal_order(self, curr, num_nodes, visited, path):
        if len(visited) == num_nodes:
            ### print("done")
            return path
        visited.add(curr)

        path.append(curr)
        for neighbor, path_to in self.edges[curr]:
            if neighbor not in visited:
                printed = True
                path = self.fast_traversal_order(neighbor, num_nodes, visited, path)

        return path
    
    def filtered_traversal_order(self):
        #full_traversal_order = self.full_traversal_order(self.start_vertex.tile_coord,self.num_tiles,set(),deque())
        full_traversal_order = self.fast_traversal_order(self.start_vertex.tile_coord,self.num_tiles,set(),deque())
        filtered_traversal_order = deque()

        visited = set()
        i = 0
        for tile in full_traversal_order:
            filtered_traversal_order.append(tile)
            visited.add(tile)
            if len(visited) == self.num_tiles:
                # print("finished!")
                break
        
        # print(len(full_traversal_order), len(filtered_traversal_order))
        if len(visited) != self.num_tiles:
            pass
            print(f"didnt visit everything {len(visited)}/{self.num_tiles}")
        return filtered_traversal_order

    # TODO: finish
    def optimized_traversal_order(self):
        filtered_traversal_order = self.filtered_traversal_order()
        self.check_tile_traversal_order(filtered_traversal_order)

        return filtered_traversal_order
        # return (None, deque())

    def check_tile_traversal_order(self, order):
        for i in range(len(order) - 1):
            a = order[i]
            b = order[i + 1]
            if manhattan(a, b) > 1:
                # print(f"traversal order is incorrect: {a} {b}")
                r_a, c_a = a
                r_b, c_b = b

                a_cell = self.vertices[r_a][c_a].cell_coord
                b_cell = self.vertices[r_b][c_b].cell_coord
                goal, path = self.cell_bfs(a_cell, b_cell)
                # print(path)

                if goal != None:
                    pass
                    # self.add_edge(a, b, path)
                # TODO: finish
        return 

In [ ]:
class BotTwo(BotOne):    
    NAME = "bot_2"
    def __init__(self, ship, k=1):
        super().__init__(ship, k=k)
        self.MST = MST(ship=ship, k=k, init_pos=self.position)
        self.traversal_order = self.MST.optimized_traversal_order()
    
    def need_to_scan(self, tile_coord):
        r_tile, c_tile = tile_coord
        r_start = r_tile * self.MST.tile_D
        c_start = c_tile * self.MST.tile_D

        # print(tile_coord)
        for r in range(self.MST.tile_D):
            for c in range(self.MST.tile_D):
                r_curr = r_start + r
                c_curr = c_start + c
                if (r_curr, c_curr) in self.unseen:
                    return True
                #print(f"({r_curr} {c_curr})", end="")
            #print("ASDF")
        # print("tile skipped")
        return False

    def phase_1(self):
        res = False
        cost = 0
        # print(len(self.traversal_order), self.traversal_order)
        if len(self.traversal_order) == 1:
            print(f"ur high {self.traversal_order}")
        if not self.future_path:
            self.sense(self.k)
            cost += 1
            if self.leaky: 
                # print(f"phase 1 leaky {self.position}, {self.leaky}")
                # print("SENSE")
                return res, cost
            else:
                A = self.traversal_order.popleft()
                B = self.traversal_order[0]
                
                while not self.need_to_scan(B):
                    self.traversal_order.popleft()
                    B = self.traversal_order[0]
                #print(self.need_to_scan(B))
                path = self.MST.get_edge(A, B)[2]
                cell_A = self.MST.get_vertex(A).cell_coord
                cell_B = self.MST.get_vertex(B).cell_coord
                goal, path = self.MST.cell_bfs(cell_A, cell_B)
                if path == None:
                    print(f"NonePath {self.position} {cell_A} {cell_B}")
                self.future_path = path
        # iterate through path[1:], checking for leak square
        self.move()
        # print(self.path)
        cost += 1

        return res, cost
        

    def phase_2(self):
        self.move()
        return False, 1

    def action(self):
        """Choose to move or sense"""
        pos = self.position
        
        if pos in self.ship.initial_leaks:
            self.ship.initial_leaks.remove(pos)
            self.sense(self.k)
            return True, 1 # NOTE: change to 0?
        elif pos in self.leaky:
            self.not_leaky.add(pos)
            self.leaky.remove(pos)

        if not self.leaky: # PHASE 1: detect leak
            res, cost = self.phase_1()
        else: # PHASE 2: find leak
            res, cost = self.phase_2()
                
        return res, cost

    def move(self):
        """Update self.position and cell enum to mark path"""
        curr = self.position
        r, c = curr
        self.path.append(curr)
        
        if not self.future_path:        
            self.future_path = self.bfs(curr, self.leaky if self.leaky else self.unseen)[1]

        # Update path to next point in path
        next_position = self.future_path.popleft()
        if manhattan(next_position, curr) > 1:
            print(f"{curr}, {next_position}")
        nr, nc = next_position
        self.position = next_position

    def simulate(self):
        success = False

        # NOTE: for bot 6, can change success to counter variable
        while not success:
            res, cost = self.action()
            if res:
                success = True

            self.timestep += cost

        return success, self.path, self.timestep
    # TODO: implement move, sense, simulate as needed

    def check_path(self):
        for i in range(len(self.path) - 1):
            a = self.path[i]
            b = self.path[i + 1]
            #if manhattan(a, b) > 1:
            #    print(f"traversal order is incorrect: {a} {b}")

In [ ]:
seed = 42
k = 4
ship = Ship(seed=seed)
bot = BotTwo(ship, k)

ship.display(bot.position)
print("Bot position: %s" % (bot.position,))
print("Leak position: %s" % (ship.initial_leaks,))

In [ ]:
vertex = []
for x in bot.MST.vertices:
    for j in x:
        if j.cell_coord != None:
            vertex.append(j.cell_coord)

# print(bot.MST.start_vertex.tile_coord, bot.MST.start_vertex.cell_coord)
# print(bot.traversal_order)
ship.display(path=vertex)

In [ ]:
# print(bot.traversal_order)

res = bot.simulate()
# print(bot.path)
for i, j in zip(bot.path[:-1], bot.path[1:]):
    if manhattan(i, j) > 1:
        pass
        # print(f"weird {i} {j}")
print(str(res[0]) + " " + str(res[2]) + " steps")

# print(len(bot.path))
bot.check_path()

ship.display(bot.path)


In [ ]:
print("100 tests:")
sum = 0

counter = 0
for seed in SEEDS: #np.random.randint(low=0, high=1000000000, size=100):
    counter += 1
    seed = int(seed)
    ship = Ship(seed)
    bot = BotTwo(ship, 4)
    res = bot.simulate()
    sum += res[2]
    print(str(seed) + ": " + str(res[0]) + " " + str(res[2]) + " steps" + "\tavg: " + str(sum/counter))

avg = sum / 100
print("\naverage: " + str(avg) + " steps")